<a href="https://colab.research.google.com/github/bsguerrabr/corretor_ortografico/blob/main/corretor_ortografico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importando os dados

In [ ]:
# baixando os arquivos 

!wget /resources/data/artigos.txt https://raw.githubusercontent.com/bsguerrabr/corretor_ortografico/main/data/artigos.txt
!wget /resources/data/palavras.txt https://raw.githubusercontent.com/bsguerrabr/corretor_ortografico/main/data/palavras.txt

In [ ]:
# lendo o arquivo do corpus textual

with open('/content/artigos.txt', 'r') as f:
  artigos = f.read()

In [ ]:
# visualizando o início do corpus textual

print(artigos[:500])

# tokenizando as palavras

In [ ]:
# instalando o nltk

!pip install nltk
import nltk
nltk.download('punkt')
print('The nltk version is {}.'.format(nltk.__version__))

In [ ]:
# função para tokenizar as palavras

def tokeniza_palavras(texto):  
  tokens = nltk.tokenize.word_tokenize(texto)

  palavras_separadas = []

  for token in tokens:
    if token.isalpha():
        palavras_separadas.append(token)
  
  return palavras_separadas

In [ ]:
# tokenizando as palavras

lista_palavras = tokeniza_palavras(artigos)
print(lista_palavras[:50])

In [ ]:
print(f'a quantidade de palavras é {len(lista_palavras)}')

# normalizando as palavras

In [ ]:
# função para normalizar as palavras

def normaliza_palavras(lista_palavras):
  lista_palavras_normalizadas = []
  
  for palavra in lista_palavras:
    lista_palavras_normalizadas.append(palavra.lower())

  return lista_palavras_normalizadas

In [ ]:
# normalizando as palavras

lista_palavras_normalizadas = normaliza_palavras(lista_palavras)
print(lista_palavras_normalizadas[:50])

# deixando apenas palavras únicas

In [ ]:
# deixando apenas palavras únicas

lista_palavras_normalizadas_unicas = set(lista_palavras_normalizadas)
print(lista_palavras_normalizadas_unicas)

In [ ]:
print(f'a quantidade de palavras normalizadas únicas é {len(lista_palavras_normalizadas_unicas)}')

# corrigindo letra a menos

In [ ]:
# função para corrigir letra a menos

def insere_letras(fatias):    
    novas_palavras = []

    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    
    for lado_esquerdo, lado_direito in fatias:
      for letra in letras:
        novas_palavras.append(lado_esquerdo + letra + lado_direito)
    
    return novas_palavras

# corrigindo letra a mais

In [ ]:
# função para corrigir letra a mais

def remove_caractere(fatias):    
    novas_palavras = []
    
    for lado_esquerdo, lado_direito in fatias:
      novas_palavras.append(lado_esquerdo + lado_direito[1:])
    
    return novas_palavras  

# corrigindo letra trocada

In [ ]:
# função para corrigir letra trocada

def troca_letra(fatias):       
    novas_palavras = []
    
    letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
    
    for lado_esquerdo, lado_direito in fatias:
      for letra in letras:
        novas_palavras.append(lado_esquerdo + letra + lado_direito[1:])
    
    return novas_palavras

# corrigindo letras invertidas

In [ ]:
# função para corrigir letra invertida

def inverte_letras(fatias):       
    novas_palavras = []
    
    for lado_esquerdo, lado_direito in fatias:      
      if len(lado_direito) > 1:
        novas_palavras.append(lado_esquerdo + lado_direito[1] + lado_direito[0] + lado_direito[2:])
    
    return novas_palavras

# gerador de palavras

In [ ]:
# função para, baseado na palavras inserida, gerar palavras tentando inserir, remover, trocar e inverter letras

def gerador_palavras(palavra):
  fatias = []
  
  for i in range(len(palavra) + 1):
    fatias.append((palavra[:i], palavra[i:]))
  
  palavras_geradas = insere_letras(fatias)
  palavras_geradas += remove_caractere(fatias)
  palavras_geradas += troca_letra(fatias)
  palavras_geradas += inverte_letras(fatias)
  
  return palavras_geradas  

In [ ]:
# verificando as palavras geradas inserindo a palavra "lgica"

palavra = 'lgica'
palavras_geradas = gerador_palavras(palavra)
print(palavras_geradas)

In [ ]:
# verificando as palavras geradas inserindo a palavra "lóigica"

palavra = 'lóigica'
palavras_geradas = gerador_palavras(palavra)
print(palavras_geradas)

# corretor

In [ ]:
# total de palavras do corpus textual

total_palavras = len(lista_palavras_normalizadas)
total_palavras

In [ ]:
# palavras mais frequentes

frequencia = nltk.FreqDist(lista_palavras_normalizadas)
frequencia.most_common(10)

In [ ]:
# função para verificar a probabilidade de uma palavra ser a palavra correta, baseado na sua frequência de uso no corpus textual

def probabilidade(palavra_gerada):
  return frequencia[palavra_gerada] / total_palavras

In [ ]:
# função para corrigir a palavra inserida

def corretor(palavra):
  palavra_correta = ''  

  # baseado na palavras inserida, gera palavras tentando inserir, remover, trocar e inverter letras
  palavras_geradas = gerador_palavras(palavra)
    
  # dentre as palavras geradas, calcula qual tem a maior probabilidade de ser a palavra correta, baseado na sua frequência de uso no corpus textual
  palavra_correta = max(palavras_geradas, key=probabilidade)  

  return palavra_correta

In [ ]:
# testando corrigir palavra com uma letra a menos

corretor('lgica')

In [ ]:
# testando corrigir palavra com uma letra a mais

corretor('lóigica')

In [ ]:
# testando corrigir palavra com uma letra trocada

corretor('lígica')

In [ ]:
# testando corrigir palavra com uma letra invertida

corretor('lgóica')

# avaliando o corretor

In [ ]:
# função para criar dados de teste

def cria_dados_teste(nome_arquivo):
  lista_palavras_teste = []
  
  f = open(nome_arquivo, 'r')
  
  for linha in f:
    correta, errada = linha.split()
    lista_palavras_teste.append((correta, errada))
  
  f.close()
  
  return lista_palavras_teste

In [ ]:
# cria dados de teste utilizando o arquivo palavras.txt
# o arquivo palavras.txt contém uma lista de conjuntos compostos pela palavra correta e pela palavra com erro de digitação

nome_arquivo = '/content/palavras.txt'
lista_palavras_teste = cria_dados_teste(nome_arquivo)
print(lista_palavras_teste)

In [ ]:
# função para avaliar o corretor ortográfico

def avaliador(lista_palavras_teste, vocabulario):
  quantidade_palavras = len(lista_palavras_teste)
  acertos = 0
  desconhecidas = 0
  
  for correta, errada in lista_palavras_teste:
    palavra_corrigida = corretor(errada)
    desconhecidas += (correta not in vocabulario)

    if (palavra_corrigida == correta): 
      acertos += 1
  
  taxa_acerto = round(((acertos * 100) / quantidade_palavras), 2)
  
  taxa_erro_por_palavras_desconhecidas = round(((desconhecidas * 100) / quantidade_palavras), 2)

  print(f'a taxa de acerto é {taxa_acerto}% de um total de {quantidade_palavras} palavras.\n' +
    f'a taxa de erro por palavras desconhecidas é {taxa_erro_por_palavras_desconhecidas}%.')

In [ ]:
# avaliando o corretor ortográfico

avaliador(lista_palavras_teste, lista_palavras_normalizadas_unicas)

# aplicando o corretor ortográfico

In [ ]:
# inserindo a palavra "lógica" com uma letra a menos: "lgica"

palavra = 'lgica'

print(corretor(palavra))

In [ ]:
# inserindo a palavra "rua" com uma letra a mais: "ruai"

palavra = 'ruai'

print(corretor(palavra))

In [ ]:
# inserindo a palavra "papel" com uma letra trocada: "popel"

palavra = 'popel'

print(corretor(palavra))

In [ ]:
# inserindo a palavra "navegador" com uma letra invertida: "navgeador"

palavra = 'navgeador'

print(corretor(palavra))

# conclusão

In [ ]:
# o corretor corrige palavras inseridas com uma letra a menos, uma letra a mais, uma letra trocada ou uma letra invertida com: 

# taxa de acerto é 76.34% de um total de 186 palavras
# taxa de erro por palavras desconhecidas é 6.99%